In [10]:
pip install dash

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/7.8 MB 2.6 MB/s eta 0:00:03
   - -------------------------------------- 0.4/7.8 MB 5.8 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/7.8 MB 5.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/7.8 MB 5.4 MB/s eta 0:00:02
   ---- ----------------------------------- 0.9/7.8 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.8 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/7.8 MB 4.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.4/7.8 MB 4.6 MB/s eta 0:00:02
   -------- ------------------------------- 1.6/7.8 MB 4.8 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/7.8 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.0/7.8 MB 5.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.2/7.8 MB 5.0 MB/s eta 0:00:02
   ----------- ----

In [32]:
df = pd.read_csv('C:/Users/Julien GILLES/Documents/prices.csv')

In [33]:
df.head(5)

,Date,Accor S.A.,Airbus,ArcelorMittal,Air Liquide,AXA S.A.,BNP Paribas,Bouygues,Bureau Veritas,Capgemini,...,Schneider Electric,Société Générale,Stellantis,STMicroelectronics,Téléperformance,Thales,TotalEnergies,WFD Unibail Rodamco,Veolia Environnement,Vinci
0,2025-03-07 16:58:34,46.31,169.42,30.58,179.38,38.66,76.41,35.56,29.10,154.00,...,221.95,41.850,11.648,23.535,102.85,239.7,56.80,75.58,30.53,117.025
1,2025-03-07 16:58:46,46.31,169.42,30.58,179.38,38.66,76.41,35.56,29.10,154.00,...,221.95,41.850,11.648,23.535,102.85,239.7,56.80,75.58,30.53,117.025
2,2025-03-07 16:59:24,46.31,169.56,30.57,179.48,38.67,76.40,35.56,29.10,153.95,...,222.00,41.875,11.658,23.560,102.95,239.8,56.86,75.60,30.54,117.000
3,2025-03-07 17:04:46,46.78,170.50,30.97,180.08,38.92,77.02,35.85,29.18,154.10,...,225.30,42.175,11.776,23.825,103.85,238.3,57.02,76.00,30.70,117.950


In [34]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Fonction pour lire le fichier CSV
def lire_csv(fichier):
    return pd.read_csv(fichier)

# Mise en page de l'application
app.layout = html.Div([
    html.H1("Prix des actions en temps réel"),
    
    # Interval pour la mise à jour automatique
    dcc.Interval(id='interval-component', interval=300000, n_intervals=0),
    
    # Affichage des derniers prix des actions
    html.Div(id='prix-actions'),

    # Graphique de l'évolution des prix
    dcc.Graph(id='graph-evolution')
])

# Callback pour mettre à jour les prix des actions
@app.callback(
    Output('prix-actions', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_prix(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Assurez-vous que le chemin est correct
        derniere_ligne = df.iloc[-1]  # Récupère la dernière ligne du DataFrame
        date = derniere_ligne['Date']
        prix = derniere_ligne[1:].to_dict()  # Supposons que la première colonne est la date

        texte = [html.P(f"Dernière mise à jour : {date}")]
        for action, valeur in prix.items():
            texte.append(html.P(f"{action}: {valeur}"))

        return texte
    except Exception as e:
        return html.P(f"Erreur lors de la mise à jour : {e}")

# Callback pour mettre à jour le graphique
@app.callback(
    Output('graph-evolution', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_graphique(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger les données du CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir la colonne date en format datetime

        # Transformation des données pour Plotly
        df_melted = df.melt(id_vars=['Date'], var_name='Entreprise', value_name='Prix')

        # Création du graphique avec Plotly Express
        fig = px.line(df_melted, x='Date', y='Prix', color='Entreprise', 
                      title="Évolution des prix des actions des 40 entreprises")

        return fig
    except Exception as e:
        return px.line(title=f"Erreur lors du chargement des données : {e}")

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True)


D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
#calcul rendements/volatilité

In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Fonction pour lire le fichier CSV
def lire_csv(fichier):
    return pd.read_csv(fichier)

# Mise en page de l'application
app.layout = html.Div([
    html.H1("Prix des actions en temps réel"),
    
    # Interval pour la mise à jour automatique (toutes les 5 minutes)
    dcc.Interval(id='interval-component', interval=300000, n_intervals=0),
    
    # Affichage des derniers prix et statistiques
    html.Div(id='prix-actions', style={'display': 'flex', 'flex-wrap': 'wrap'}),

    # Graphique de l'évolution des prix
    dcc.Graph(id='graph-evolution')
])

# Callback pour mettre à jour les prix et statistiques des actions
@app.callback(
    Output('prix-actions', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_prix(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir en format datetime

        derniere_ligne = df.iloc[-1]  # Derniers prix disponibles
        date = derniere_ligne['Date']
        prix_actuels = derniere_ligne[1:].to_dict()  # Prix des actions

        # Calcul du rendement et de la volatilité
        cartes = [html.P(f"Dernière mise à jour : {date}", style={'width': '100%'})]
        
        for action in prix_actuels.keys():
            historique = df[['Date', action]].dropna()  # Éviter les valeurs manquantes
            rendement = ((historique[action].iloc[-1] - historique[action].iloc[0]) / historique[action].iloc[0]) * 100
            historique['retours'] = historique[action].pct_change()  # Rendements quotidiens
            volatilite = historique['retours'].std() * 100  # Convertir en %

            cartes.append(
                html.Div([
                    html.H3(action, style={'margin-bottom': '5px'}),
                    html.P(f"Prix : {prix_actuels[action]:.2f} €"),
                    html.P(f"Rendement : {rendement:.2f} %"),
                    html.P(f"Volatilité : {volatilite:.2f} %")
                ], style={
                    'border': '1px solid black',
                    'border-radius': '10px',
                    'padding': '10px',
                    'margin': '10px',
                    'width': '200px',
                    'text-align': 'center',
                    'background-color': '#f4f4f4'
                })
            )

        return cartes
    except Exception as e:
        return html.P(f"Erreur lors de la mise à jour : {e}")

# Callback pour mettre à jour le graphique
@app.callback(
    Output('graph-evolution', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_graphique(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir la colonne date en format datetime

        # Transformation des données pour Plotly
        df_melted = df.melt(id_vars=['Date'], var_name='Entreprise', value_name='Prix')

        # Création du graphique avec Plotly Express
        fig = px.line(df_melted, x='Date', y='Prix', color='Entreprise', 
                      title="Évolution des prix des actions des 40 entreprises")

        return fig
    except Exception as e:
        return px.line(title=f"Erreur lors du chargement des données : {e}")

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
#calcul ratio de Sharpe

In [34]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Taux sans risque (2% annuel par exemple)
TAUX_SANS_RISQUE = 0.02 / 252

# Fonction pour lire le fichier CSV
def lire_csv(fichier):
    return pd.read_csv(fichier)

# Mise en page de l'application
app.layout = html.Div([
    html.H1(
        "Cours actions CAC 40",
        style={
            'textAlign': 'center',  # Centre le texte
            'fontFamily': 'Arial, sans-serif',  # Change la police
            'color': 'white',  # Change la couleur du texte
            'fontSize': '32px',  # Change la taille de la police
            'fontWeight': 'bold'  # Change l'épaisseur de la police
        }
    ),

    html.Img(
    src='/assets/logo.webp',  # Utilisation du chemin relatif à partir du dossier 'assets'
    style={
        'position': 'absolute',
        'top': '10px',
        'right': '10px',
        'width': '80px',
        'height': '80px'
    }
    ),

    
    # Interval pour la mise à jour automatique (toutes les 5 minutes)
    dcc.Interval(id='interval-component', interval=300000, n_intervals=0),
    
    # Affichage des derniers prix et statistiques
    html.Div(id='prix-actions', style={'display': 'flex', 'flex-wrap': 'wrap'}),

    # Graphique de l'évolution des prix
    dcc.Graph(id='graph-evolution')
], style={'backgroundColor': '#001F3F', 'padding': '20px', 'min-height': '100vh'})  # Bleu marine

# Callback pour mettre à jour les prix et statistiques des actions
@app.callback(
    Output('prix-actions', 'children'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_prix(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir en format datetime

        derniere_ligne = df.iloc[-1]  # Derniers prix disponibles
        date = derniere_ligne['Date']
        prix_actuels = derniere_ligne[1:].to_dict()  # Prix des actions

        # Calcul du rendement, volatilité et Sharpe Ratio
        cartes = [html.P(f"Dernière mise à jour : {date}", style={'width': '100%', 'color': 'white'})]
        
        for action in prix_actuels.keys():
            historique = df[['Date', action]].dropna()  # Éviter les valeurs manquantes
            rendement = ((historique[action].iloc[-1] - historique[action].iloc[0]) / historique[action].iloc[0]) * 100
            historique['retours'] = historique[action].pct_change()  # Rendements quotidiens
            volatilite = historique['retours'].std() * 100  # Convertir en %
            rendement_moyen = historique['retours'].mean()  # Moyenne des rendements journaliers
            
            # Calcul du Sharpe Ratio
            sharpe_ratio = (rendement_moyen - TAUX_SANS_RISQUE) / historique['retours'].std() if historique['retours'].std() != 0 else 0

            cartes.append(
                html.Div([
                    html.H3(action, style={'margin-bottom': '5px', 'color': 'white'}),
                    html.P(f"Prix : {prix_actuels[action]:.2f} €", style={'color': 'white'}),
                    html.P(f"Rendement : {rendement:.2f} %", style={'color': 'white'}),
                    html.P(f"Volatilité : {volatilite:.2f} %", style={'color': 'white'}),
                    html.P(f"Sharpe Ratio : {sharpe_ratio:.2f}", style={'color': 'white'})
                ], style={
                    'border': '1px solid white',
                    'border-radius': '10px',
                    'padding': '10px',
                    'margin': '10px',
                    'width': '250px',
                    'text-align': 'center',
                    'background-color': '#003366'
                })
            )

        return cartes
    except Exception as e:
        return html.P(f"Erreur lors de la mise à jour : {e}", style={'color': 'red'})

# Callback pour mettre à jour le graphique
@app.callback(
    Output('graph-evolution', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_graphique(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir la colonne date en format datetime

        # Transformation des données pour Plotly
        df_melted = df.melt(id_vars=['Date'], var_name='Entreprise', value_name='Prix')

        # Création du graphique avec Plotly Express
        fig = px.line(df_melted, x='Date', y='Prix', color='Entreprise', 
                      title="Évolution des prix des actions des 40 entreprises")
        fig.update_layout(plot_bgcolor='#001F3F', paper_bgcolor='#001F3F', font=dict(color='white'))

        return fig
    except Exception as e:
        return px.line(title=f"Erreur lors du chargement des données : {e}")

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.



D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: Fut

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.



D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
# TEST 09/03/25

In [26]:
pip install dash-table


Note: you may need to restart the kernel to use updated packages.


In [31]:
pip install colorthief

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: Fut

Note: you may need to restart the kernel to use updated packages.


D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [44]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_table
import base64

# Initialiser l'application Dash
app = dash.Dash(__name__)

# Taux sans risque (2% annuel par exemple)
TAUX_SANS_RISQUE = 0.02 / 252

# Fonction pour lire le fichier CSV
def lire_csv(fichier):
    return pd.read_csv(fichier)

# Encoder l'image en base64
with open('C:/Users/Julien GILLES/Documents/GitHub/Python-Git-Linux_Final-Project/assets/logo.webp', 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

# Mise en page de l'application
app.layout = html.Div([
    html.H1(
        "Cours actions CAC 40",
        style={
            'textAlign': 'center',  # Centre le texte
            'fontFamily': 'Arial',  # Police Arial
            'color': 'white',  # Change la couleur du texte
            'fontSize': '32px',  # Change la taille de la police
            'fontWeight': 'bold'  # Change l'épaisseur de la police
        }
    ),

    html.Img(
        src=f'data:image/webp;base64,{encoded_image}',  # Utilisation de l'image encodée en base64
        style={
            'position': 'absolute',
            'top': '10px',
            'right': '10px',
            'width': '80px',
            'height': '80px'
        }
    ),

    # Interval pour la mise à jour automatique (toutes les 5 minutes)
    dcc.Interval(id='interval-component', interval=300000, n_intervals=0),

    # Affichage des derniers prix et statistiques dans un tableau
    dash_table.DataTable(
        id='table-actions',
        columns=[
            {"name": "Action", "id": "Action"},
            {"name": "Prix (€)", "id": "Prix", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "Rendement (%)", "id": "Rendement", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "Volatilité (%)", "id": "Volatilité", "type": "numeric", "format": {"specifier": ".2f"}},
            {"name": "Sharpe Ratio", "id": "Sharpe Ratio", "type": "numeric", "format": {"specifier": ".2f"}}
        ],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'center',
            'color': 'white',
            'fontFamily': 'Arial'  # Police Arial
        },
        style_header={
            'backgroundColor': '#003366',
            'color': 'white',  # Mettre le texte en gras
            'fontFamily': 'Arial'  # Police Arial
        },
        style_data={
            'backgroundColor': '#001F3F',
            'fontFamily': 'Arial'  # Police Arial
        }
    ),

    # Graphique de l'évolution des prix
    dcc.Graph(id='graph-evolution')
], style={
    'backgroundColor': '#001F3F', 
    'padding': '20px', 
    'min-height': '100vh',
    'fontFamily': 'Arial'  # Police Arial pour toute la page
})

# Callback pour mettre à jour les prix et statistiques des actions
@app.callback(
    Output('table-actions', 'data'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_prix(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir en format datetime

        derniere_ligne = df.iloc[-1]  # Derniers prix disponibles
        date = derniere_ligne['Date']
        prix_actuels = derniere_ligne[1:].to_dict()  # Prix des actions

        # Calcul du rendement, volatilité et Sharpe Ratio
        data = []

        for action in prix_actuels.keys():
            historique = df[['Date', action]].dropna()  # Éviter les valeurs manquantes
            rendement = ((historique[action].iloc[-1] - historique[action].iloc[0]) / historique[action].iloc[0]) * 100
            historique['retours'] = historique[action].pct_change()  # Rendements quotidiens
            volatilite = historique['retours'].std() * 100  # Convertir en %
            rendement_moyen = historique['retours'].mean()  # Moyenne des rendements journaliers

            # Calcul du Sharpe Ratio
            sharpe_ratio = (rendement_moyen - TAUX_SANS_RISQUE) / historique['retours'].std() if historique['retours'].std() != 0 else 0

            data.append({
                "Action": action,
                "Prix": prix_actuels[action],
                "Rendement": rendement,
                "Volatilité": volatilite,
                "Sharpe Ratio": sharpe_ratio
            })

        return data
    except Exception as e:
        return [{"Action": "Erreur", "Prix": 0, "Rendement": 0, "Volatilité": 0, "Sharpe Ratio": 0}]

# Callback pour mettre à jour le graphique
@app.callback(
    Output('graph-evolution', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def mettre_a_jour_graphique(n):
    try:
        df = lire_csv('C:/Users/Julien GILLES/Documents/prices.csv')  # Charger le CSV
        df['Date'] = pd.to_datetime(df['Date'])  # Convertir la colonne date en format datetime

        # Transformation des données pour Plotly
        df_melted = df.melt(id_vars=['Date'], var_name='Entreprise', value_name='Prix')

        # Création du graphique avec Plotly Express
        fig = px.line(df_melted, x='Date', y='Prix', color='Entreprise',
                      title="Évolution des prix des actions du CAC 40")
        fig.update_layout(plot_bgcolor='#001F3F', paper_bgcolor='#001F3F', font=dict(color='white'))

        return fig
    except Exception as e:
        return px.line(title=f"Erreur lors du chargement des données : {e}")

# Exécuter l'application
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)


D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

D:\Anaconda\Lib\site-packages\_plotly_utils\basevalidators.py:106: Fut